In [1]:
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import serialization
import os

# --- PART 1: Certificate Authority (CA) Simulation ---
def generate_certificate(public_key_pem: bytes, identity: str) -> dict:
    """
    Simulate issuing an X.509 certificate.

    Args:
        public_key_pem (bytes): The PEM-encoded ECC public key.
        identity (str): The identity of the certificate owner (e.g., "Bob").

    Returns:
        dict: A simulated certificate containing the identity, public key, and signature.

    TODO:
    1. Hash the identity and public key together using SHA-256.
    2. Simulate a "digital signature" using the hash.
    3. Return the simulated certificate as a dictionary.
    """
    # Your Code Here (TODO)
    return None  # Replace with generated certificate


def verify_certificate(certificate: dict) -> bool:
    """
    Simulate verifying a CA-issued certificate.

    Args:
        certificate (dict): The simulated certificate containing identity, public key, and signature.

    Returns:
        bool: True if the certificate is valid, False otherwise.

    TODO:
    1. Recompute the hash using the stored identity and public key.
    2. Compare the computed hash with the stored signature.
    """
    # Your Code Here (TODO)
    return False  # Replace with correct verification

# --- PART 2: ECC Key Exchange (Server Side - Bob) ---
def generate_ecc_keys() -> tuple:
    """
    Generate an ECC key pair using SECP256R1.

    Returns:
        tuple: A tuple containing:
            - private_key (EllipticCurvePrivateKey): The generated private key.
            - public_pem (bytes): The PEM-encoded public key.

    TODO:
    1. Generate an ECC private key using SECP256R1.
    2. Extract the corresponding public key.
    3. Serialize the public key into PEM format.
    """
    # Your Code Here (TODO)
    return None, None  # Replace with private key and public PEM

# --- PART 3: Shared Key Derivation (Client Side - Alice) ---
def derive_shared_key(private_key, peer_public_key_pem: bytes) -> bytes:
    """
    Compute the shared key using ECDH and derive an AES session key using HKDF.

    Args:
        private_key (EllipticCurvePrivateKey): The ECC private key.
        peer_public_key_pem (bytes): The PEM-encoded public key of the peer.

    Returns:
        bytes: A 32-byte AES session key.

    TODO:
    1. Deserialize the peer's public key.
    2. Compute the shared secret using ECDH.
    3. Derive a 32-byte AES key using HKDF (SHA-256).
    """
    # Your Code Here (TODO)
    return None  # Replace with derived AES key

# --- PART 4: Secure Data Transmission (Client Side - Alice) ---
def encrypt_message(message: str, aes_key: bytes) -> tuple:
    """
    Encrypt a message using AES-GCM.

    Args:
        message (str): The plaintext message to encrypt.
        aes_key (bytes): A 32-byte AES key.

    Returns:
        tuple: A tuple containing:
            - iv (bytes): A 12-byte randomly generated Initialization Vector.
            - ciphertext (bytes): The encrypted message.
            - tag (bytes): The authentication tag for integrity verification.

    TODO:
    1. Generate a random 12-byte IV.
    2. Encrypt the message using AES-GCM.
    3. Return the IV, ciphertext, and tag.
    """
    # Your Code Here (TODO)
    return None, None, None  # Replace with IV, ciphertext, and tag

# --- PART 5: Decryption and Verification (Server Side - Bob) ---
def decrypt_message(iv: bytes, ciphertext: bytes, tag: bytes, aes_key: bytes) -> str:
    """
    Decrypt an AES-GCM encrypted message.

    Args:
        iv (bytes): The Initialization Vector.
        ciphertext (bytes): The encrypted message.
        tag (bytes): The authentication tag.
        aes_key (bytes): The AES key used for decryption.

    Returns:
        str: The decrypted plaintext message.

    TODO:
    1. Decrypt the message using AES-GCM.
    2. Return the plaintext message.
    """
    # Your Code Here (TODO)
    return None  # Replace with decrypted message

# --- Interactive TLS Handshake Simulation ---
def main():
    print("\n🔐 Simulating Secure TLS Communication...\n")

    # Bob generates a long-term ECC key pair used ONLY for authentication (certificate + signing)
    print("[Server] Generating Bob's certificate (signing) key pair...\n")
    server_cert_private_key, server_cert_public_pem = generate_ecc_keys()

    # CA issues a certificate for Bob (reused across connections)
    print("[CA] Issuing a certificate for Bob...\n")
    certificate = generate_certificate(server_cert_public_pem, "Bob")

    conn = 1
    while True:
        print(f"\n--- New Connection #{conn} ---\n")

        # Alice verifies Bob’s certificate (each connection)
        print("[Client] Verifying Bob’s certificate...\n")
        if verify_certificate(certificate):
            print("[Client] ✅ Certificate verified! Proceeding with secure communication.\n")
        else:
            print("[Client] ❌ Certificate verification failed! Exiting.\n")
            return

        # --- Ephemeral ECDH key exchange ---
        # Fresh ECDH parameters per connection; authenticated by Bob's certificate key
        print("[Server] Generating ephemeral ECDH key pair for this connection...\n")
        server_eph_private_key, server_eph_public_pem = generate_ecc_keys()

        # Bob signs the ephemeral parameters (like TLS ServerKeyExchange)
        server_signature = server_cert_private_key.sign(
            server_eph_public_pem,
            ec.ECDSA(hashes.SHA256())
        )

        # Alice verifies the signature using Bob's certified public key
        print("[Client] Verifying server signature on ephemeral parameters...\n")
        bob_cert_pub = serialization.load_pem_public_key(certificate["public_key"])
        try:
            bob_cert_pub.verify(server_signature, server_eph_public_pem, ec.ECDSA(hashes.SHA256()))
            print("[Client] ✅ Ephemeral parameters authenticated.\n")
        except Exception:
            print("[Client] ❌ Ephemeral parameters NOT authenticated! Exiting.\n")
            return

        # Alice generates fresh ECDH key pair for this connection
        print("[Client] Generating ephemeral ECDH key pair...\n")
        client_private_key, client_public_pem = generate_ecc_keys()

        # Alice derives AES session key using Bob's ephemeral public key
        print("[Client] Deriving shared AES session key...\n")
        aes_key = derive_shared_key(client_private_key, server_eph_public_pem)

        # Alice encrypts her message
        message = input("[Client] 📝 Enter a secure message to send to Bob: ")
        iv, ciphertext, tag = encrypt_message(message, aes_key)

        # Bob derives the same AES session key using Alice's ephemeral public key
        print("\n[Server] 🔓 Deriving shared AES session key...\n")
        server_aes_key = derive_shared_key(server_eph_private_key, client_public_pem)

        # Bob decrypts the message
        print("\n[Server] 🔓 Decrypting message...\n")
        decrypted_message = decrypt_message(iv, ciphertext, tag, server_aes_key)
        print("\n📖 Decrypted Message:", decrypted_message.decode() if decrypted_message else "Decryption Failed")

        print("\n✅ Connection complete. Tearing down session state...\n")

        conn += 1
        again = input("Start another connection? (y/n): ").strip().lower()
        if again != 'y':
            break

    print("\n✅ TLS Handshake Simulation Finished!")

if __name__ == "__main__":
    main()



🔐 Simulating Secure TLS Communication...

[Server] Generating Bob's certificate (signing) key pair...

[CA] Issuing a certificate for Bob...


--- New Connection #1 ---

[Client] Verifying Bob’s certificate...

[Client] ❌ Certificate verification failed! Exiting.

